In [1]:
from sklearn.metrics import classification_report

In [2]:
import onnx
import onnxruntime
import idx2numpy
import numpy as np

In [3]:
sess_options = onnxruntime.SessionOptions()
sess = onnxruntime.InferenceSession('fashion_mnist.onnx', sess_options)

In [4]:
input_name = sess.get_inputs()[0].name
print("input name", input_name)
input_shape = sess.get_inputs()[0].shape
print("input shape", input_shape)
input_type = sess.get_inputs()[0].type
print("input type", input_type)

input name input
input shape ['batch_size', 1, 28, 28]
input type tensor(float)


In [5]:
output_name = sess.get_outputs()[0].name
print("output name", output_name)
output_shape = sess.get_outputs()[0].shape
print("output shape", output_shape)
output_type = sess.get_outputs()[0].type
print("output type", output_type)

output name output
output shape ['batch_size', 10]
output type tensor(float)


In [6]:
test_images = idx2numpy.convert_from_file('data/raw/t10k-images-idx3-ubyte')
test_labels = idx2numpy.convert_from_file('data/raw/t10k-labels-idx1-ubyte')

In [9]:
batch_inf_imgs = []
for test_image in test_images:
    test_image = test_image / 255.
    test_image = test_image.astype(np.float32)
    test_image = np.expand_dims(test_image, axis=0)
    batch_inf_imgs.append(test_image)

batch_inf_imgs = np.array(batch_inf_imgs)

In [15]:
results = sess.run([output_name], {input_name: batch_inf_imgs})[0]

In [16]:
y_pred = np.argmax(results, 1)
y_true = test_labels

In [17]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.82      0.85      1000
           1       0.99      0.97      0.98      1000
           2       0.86      0.85      0.86      1000
           3       0.89      0.92      0.90      1000
           4       0.82      0.89      0.85      1000
           5       0.98      0.98      0.98      1000
           6       0.73      0.72      0.72      1000
           7       0.94      0.97      0.96      1000
           8       0.98      0.98      0.98      1000
           9       0.98      0.95      0.96      1000

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000

